### Домашнее задание 2. Извлечение коллокаций + NER

Импортируем нужные модули и скачиваем данные с отзывами. Я выбрала категорию товаров, связанных с видеоиграми:

In [ ]:
import gzip
import pandas as pd
from tqdm import tqdm
from urllib.request import urlopen
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz

--2022-12-17 17:48:54--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113104579 (108M) [application/x-gzip]
Saving to: ‘reviews_Video_Games_5.json.gz’

reviews_Video_Games 100%[===================>] 107.86M  20.4MB/s    in 10s     

2022-12-17 17:49:04 (10.8 MB/s) - ‘reviews_Video_Games_5.json.gz’ saved [113104579/113104579]



Открываем данные:

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in tqdm(g):
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Video_Games_5.json.gz')
df

231780it [00:21, 11002.84it/s]


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1.0,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4.0,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1.0,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3.0,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,DIRT 3,1308009600,"06 14, 2011"
...,...,...,...,...,...,...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,Frustrated gamer,"[0, 1]",Funny people on here are rating sellers that a...,5.0,this is for rating the system not the seller,1405814400,"07 20, 2014"
231776,A3VVMIMMTYQV5F,B00KHECZXO,Johnny Saigon,"[8, 11]",All this is is the Deluxe 32GB Wii U with Mari...,1.0,Get the Other Bundle Which Includes Extra Whee...,1403308800,"06 21, 2014"
231777,A1DD4B97M4DUC5,B00KHECZXO,migit,"[62, 66]",The package should have more red on it and sho...,1.0,Fake bundle,1401321600,"05 29, 2014"
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,"Philip Brown ""Philip & Chana""","[33, 36]",Can get this at Newegg for $329.00 and the pac...,1.0,Looks Like We Have Gougers Again.,1401667200,"06 2, 2014"


### Способы для нахождения товаров в тексте

- Сначала мне в голову пришел самый простой способ, заключающийся в поиске коллокаций с сочетаниями определенных частей речи. Например, можно предположить, что в отзывах будет часто встречаться ADJ + NOUN (признак товара и сам товар). Очевидный минус подохода — в списке коллокаций будет много лишних сочетаний, которые никак не связаны с товаром.
- Синтаксическая проверка. Например, мне кажется, что в отзывах существительные, содержащие товары, могут часто встречаться в качестве прямого объекта глагола. Однако здесь опять скорее всего будут попадаться ненужные коллокации, кроме того, из-за такой проверки в список могут не попасть и нужные вещи (например, если товар выражен субъектом в *товар пришел*)
- Наиболее логичным я считаю использование W2V. Можно обучить модель на отзывах о товарах, подобрать наиболее близкие по значению слова к существительным, которые точно имеют отношение к категории (в нашем случае удобно то, что в самой категории уже есть сочетание *video game*, которое можно записать как *videogame*, *video_game* или же просто *game*, к этим вариантам как раз можно подобрать близкие слова). Далее в отзывах можно искать коллокации уже со словами из полученных списков. В данной тетрадке я буду реализовывать именно этот вариант.

### W2V

Немного обработаем данные. Я решила убрать знаки препинания, а также заменить video game(s) на video_game(s), чтобы в модели данное сочетание сохранилось в виде одного токена. Пробел после game(s) оставила для того, чтобы в один токен не объеденились коллокации с gamer(s)

In [ ]:
def preprocess(text):
    clean_text = re.sub(r"[^\w\s]", " ", text.lower())
    clean_text = clean_text.replace("video game ", "video_game ")
    clean_text = clean_text.replace("video games ", "video_games ")
    clean_text = clean_text.replace("  ", " ")
    return clean_text

df["reviewText"] = df["reviewText"].apply(preprocess)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",installing the game was a struggle because of ...,1.0,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",if you like rally cars get this game you will ...,4.0,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1.0,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]",i got this version instead of the ps3 version ...,3.0,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",i had dirt 2 on xbox 360 and it was an okay ga...,4.0,DIRT 3,1308009600,"06 14, 2011"
...,...,...,...,...,...,...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,Frustrated gamer,"[0, 1]",funny people on here are rating sellers that a...,5.0,this is for rating the system not the seller,1405814400,"07 20, 2014"
231776,A3VVMIMMTYQV5F,B00KHECZXO,Johnny Saigon,"[8, 11]",all this is is the deluxe 32gb wii u with mari...,1.0,Get the Other Bundle Which Includes Extra Whee...,1403308800,"06 21, 2014"
231777,A1DD4B97M4DUC5,B00KHECZXO,migit,"[62, 66]",the package should have more red on it and sho...,1.0,Fake bundle,1401321600,"05 29, 2014"
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,"Philip Brown ""Philip & Chana""","[33, 36]",can get this at newegg for 329 00 and the pack...,1.0,Looks Like We Have Gougers Again.,1401667200,"06 2, 2014"


Запишем отзывы в текстовый файл, нужный для обучения модели:

In [ ]:
revs_text = " ".join([rev for rev in df['reviewText']])

In [ ]:
with open("reviews.txt", "w", encoding = "utf-8") as f:
    f.write(revs_text)

Модель:

In [ ]:
import gensim
from gensim.models import word2vec

В закомментированной части кода модель обучается, а результат записывается в файл *reviews_model*. Сделала я это просто для удобства, чтобы при запуске тетрадки не приходилось каждый раз заново обучать модель.

In [ ]:
# fl = "reviews.txt"
# data = gensim.models.word2vec.LineSentence(fl)
# model = gensim.models.Word2Vec(data, vector_size=200, window=5, min_count=10)
# model.wv.save_word2vec_format('/content/drive/MyDrive/reviews_model.txt', binary=False)
model_ = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/reviews_model.txt", binary=False)

Итак, я подбираю наиболее близкие слова к video_game, game, videogame и к этим же словам, но во множественном числе, т.к. лемматизация не выполнялась. Смотрю на топ-20, на отбираю только те, чье расстояние не ниже 0.5. Также решила убрать местоимение *it*, чтобы отбирались более осмысленные коллокации:

In [ ]:
all_sim_words = [i[0] for i in model_.most_similar(positive=["video_game"], topn=20) if i[1] > 0.5] + [i[0] for i in model_.most_similar(positive=["video_games"], topn=20) if i[1] > 0.5] + [i[0] for i in model_.most_similar(positive=["game"], topn=20) if i[1] > 0.5] + [i[0] for i in model_.most_similar(positive=["games"], topn=20) if i[1] > 0.5] + [i[0] for i in model_.most_similar(positive=["videogame"], topn=20) if i[1] > 0.5] + [i[0] for i in model_.most_similar(positive=["videogames"], topn=20) if i[1] > 0.5]
set_words = list(set(all_sim_words)) #убираем дубликаты
set_words.remove("it")
set_words

['product',
 'compilations',
 'titles',
 'mmos',
 'franchises',
 'gaming',
 'videogames',
 'fpss',
 'classics',
 'campaign',
 'crpg',
 'jrpg',
 'ffs',
 'gameplay',
 'greats',
 'video_games',
 'iterations',
 'rpgs',
 'games',
 'offerings',
 'remakes',
 'installments',
 'castlevanias',
 'hollywood',
 'mmorpgs',
 'video_game',
 'platformers',
 'videogaming',
 'exclusives',
 'videogame',
 'shooters',
 'jrpgs',
 'versions',
 'title']

Получение коллокаций *сосед справа + слово* и *слово + сосед слева*:

In [ ]:
cols = []
for rev in df['reviewText']:
    spl_rev = rev.split()
    for word in spl_rev:
        if word in set_words:
            word_ind = spl_rev.index(word)
            if word == spl_rev[0]:
                right_n = spl_rev[word_ind+1]
            elif word == spl_rev[-1]:
                left_n = spl_rev[word_ind-1]
            else:
                right_n = spl_rev[word_ind+1]
                left_n = spl_rev[word_ind-1]
            cols.append([word, right_n])
            cols.append([left_n, word])

При первых запусках кода попадалось много сочетаний со стопсловами. Изначально пробовала избавляться от них еще перед обучением модели, однако, после этого модель работала хуже. Связываю я это с тем, что удаление стопслов приводило к сокращению контекстов. Поэтому я решила удалять стопслова (а точнее коллокации с ними лишь на данном этапе)

In [ ]:
from tqdm import tqdm

In [ ]:
stops = stopwords.words('english')
clean_cols = []
for col in tqdm(cols):
    if len(set(col).intersection(stops)) == 0:
        clean_cols.append(col)

100%|██████████| 728574/728574 [00:02<00:00, 260367.53it/s]


### Смотрим на метрики (T-критерий Стъюдента, мера Дайса, PMI):

In [ ]:
metrics = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(clean_cols)
finder.apply_freq_filter(10)

In [ ]:
t = finder.score_ngrams(metrics.student_t)
dice = finder.score_ngrams(metrics.dice)
pmi = finder.score_ngrams(metrics.pmi)


df_results = pd.DataFrame({'T-score': t[:50], "Dice": dice[:50], 'PMI': pmi[:50]})
df_results

,T-score,Dice,PMI
0,"((player, campaign), 48.968256736045326)","((person, shooters), 0.4827586206896552)","((mmorpgs, massively), 11.470344248338646)"
1,"((games, like), 47.72215933761145)","((player, campaign), 0.3178891554096705)","((formidable, crpg), 10.912507723594327)"
2,"((mini, games), 46.59720951927893)","((campaign, mode), 0.17215880537009118)","((wasteland, crpg), 10.649473317760533)"
3,"((person, shooters), 44.14725845193336)","((previous, installments), 0.15107142857142858)","((crpg, collector), 10.050011247344262)"
4,"((gaming, experience), 38.39341333390285)","((previous, versions), 0.1200885445489762)","((2011, crpg), 10.038038605678185)"
5,"((best, games), 36.672785202298115)","((play, video_games), 0.11918203487642705)","((hollywood, sign), 9.534624999670909)"
6,"((two, games), 34.31097039435096)","((gaming, experience), 0.11782542776694824)","((hollywood, actors), 9.224669546449622)"
7,"((campaign, mode), 33.88964248127012)","((hd, remakes), 0.117096018735363)","((hd, remakes), 8.770923023032575)"
8,"((fighting, games), 32.58539134454901)","((playing, video_games), 0.11334691798267957)","((hollywood, blockbuster), 8.502203521978533)"
9,"((racing, games), 31.616355875079122)","((console, versions), 0.11159990738596898)","((hollywood, film), 8.431120423917049)"


На мой взгляд, везде получились довольно хорошие результаты. Если смотреть на самые первые значения выдач, то можно было бы сказать, что лучше всего себя показала мера Дайса (в t-score вторым словосочетанием идет довольно бесмысленная сама по себе биграмма *games like*, в pmi первым вариантом — *mmorpgs massively*). Впрочем, это не отменяет то, что разннобразие коллокаций и товаров в них можно увидеть как в результатах меры Дайса, так и в PMI. В критерии Стъюдента большая часть коллокаций включает в себя game(s), из-за чего выдача кажется более монотонной на фоне двух других метрик, однако полученные коллокации выглядят хорошо.

In [ ]:
str_cols = [" ".join(col) for col in clean_cols]

In [ ]:
print('videogame')
print('---')
for col in set(str_cols):  
    if "videogame" in col:
        print(col)

print('shooter')
print('---')
for col in set(str_cols):  
    if "shooter" in col:
        print(col)

print('campaign')
print('---')
for col in set(str_cols):  
    if "campaign" in col:
        print(col)

print('version')
print('---')
for col in set(str_cols):  
    if "remake" in col:
        print(col)

print('installment')
print('---')
for col in set(str_cols):  
    if "installment" in col:
        print(col)

videogame
---
love videogames
free videogames
fighting videogames
videogame party
videogame go
crazy videogames
videogame variety
masterful videogame
sports videogame
videogame meaning
rewarding videogame
videogame hell
beautiful videogames
videogames com
videogame adaptation
like videogame
gameplay videogames
school videogame
videogame instead
became videogame
unique videogame
fresh videogame
videogames n64
2009 videogame
past videogame
fried videogame
local videogame
videogame need
legit videogame
sandbox videogame
perfect videogame
videogames anymore
prettiest videogames
animated videogame
responsibilities videogames
two videogame
plays videogames
videogames whether
espn videogame
videogames sequels
videogame controller
videogames go
videogame gauntlet
collection videogame
videogame porn
videogame license
impressive videogame
videogames count
videogames become
average videogame
34 videogame
videogame surpasses
certain videogame
videogames saga
videogame space
videogame enthusiasts
s